## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [17]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
print(data.head(4))# выводи первые 4-е строки таблицы data
print('------------------------')
print('ОБЩАЯ ИНФОРМАЦИЯ О ТАБЛИЦЕ')
data.info() #просмотр общей информации о таблице
#print(data.isnull().sum())
#print('-------------------------')
#print(data[data['days_employed'].isnull()].head()) #выводим нулевые значения по столбцу 'days_employed' в начало таблицы
#print('------------------------')
#print(data[data['total_income'].isnull()].head()) #выводим нулевые значения по столбцу 'days_employed' в начало таблицы 'total_income'

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   

     family_status  family_status_id gender income_type  debt   total_income  \
0  женат / замужем                 0      F   сотрудник     0  253875.639453   
1  женат / замужем                 0      F   сотрудник     0  112080.014102   
2  женат / замужем                 0      M   сотрудник     0  145885.952297   
3  женат / замужем                 0      M   сотрудник     0  267628.550329   

                      purpose  
0               покупка жилья  
1     приобретение автомобиля  
2               покупка жилья  
3  дополнительное образование  
------------------------
ОБЩАЯ ИНФОРМАЦИЯ О ТАБЛИЦЕ
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entri

### Вывод

Таблица содержите 21525 и 12 столбоцов. В столбцах 'total_income' и 'days_employed' содержаться нулевые значения, в столбце 'days_employed' присутствуют отрицательные значения.

### Шаг 2. Предобработка данных

Обработка пропусков

In [18]:
data.loc[data['total_income'] == 'None', 'total_income'] = 'NaN' #удаляем пропуски None с помощью loc, заменяя их на NaN
data = data.fillna(0) #удаляем пропуски NaN с помощью fillna, заменяя на 0
data.isnull().sum() # проверяем остались ли пропуски

#ОБРАБОТКА ДАННЫХ ПО ДОХОДУ КДИЕНТА
# Обработаем пропущенные значения в столбце с получаемым доходом 'total_income'. Если доход у пенсионера
# равен нулю в таблице равен нулю, поэтому заполним такие значения константой со средней пенсией.
#В случае если у компаньона, сотрудника, госслужащий доход равен нулю, тогда это значеие заменим на рассчитанное по медиане.
data['total_income'] = data['total_income'].astype('int') 
pencia = 14924 #константа со средней пенсией в России в 2020 году
income_median = data['total_income'].median()
income_without = 0
stipendia = 2980 #Средняя стипендия в Россси в 2020 году
income_decret = 27985 #Максимальная выплата по уходу за ребенком 
print(data['income_type'].value_counts())
print('')
print('Медиана дохода:', income_median)
print('')
def income_nan_f(row):
    income_type_nan = row['income_type']
    income_nan = row['total_income']
    if income_nan == 0:
        if income_type_nan == 'пенсионер':
            return pencia
    if income_nan == 0:
        if income_type_nan == 'компаньон':
            return income_median 
    if income_nan == 0:
        if income_type_nan == 'госслужащий':
            return income_median 
    if income_nan == 0:
        if income_type_nan == 'сотрудник':
            return income_median 
    if income_nan == 0:
        if income_type_nan == 'предприниматель':
            return income_median
    if income_nan == 0:
        if income_type_nan == 'безработный':
            return income_without
    if income_nan == 0:
        if income_type_nan == 'студент':
            return stipendia
    if income_nan == 0:
        if income_type_nan == 'в декрете':
            return income_decret 

#ОБРАБОТКА ДАННЫХ ПО КОЛИЧЕСТВУ ДЕТЕЙ
print('---------------')
print('Кол-во детей по возрастам до обработки неверных значений:')
print(data['children'].value_counts())
data['children'] = data['children'].replace(20, 2) # в таблице присутствуют значение 20 детей в семье, таких позиций 76, поэтому считаю это опечаткой, т.е. хотели указать 2, а получилось 20, в свзяи с чем меням 20 на 2
data['children'] = data['children'].replace(-1, 1) # изменям кол-во детей с -1 на 1, т.к. это опечатка 
print('---------------')
print('Кол-во детей по возрастам после обработки неверных значений:')
print(data['children'].value_counts())

#ОБРАБОТКА ДАННЫХ ПО ТРУДОВОМУ СТАЖУ
#Напишем функцию, которая изменяет отрицательное значение трудового стажа на положительное,
# т.к. это не корректное представление данных в столбце 'days_employed'

constant = -1
def days_employed_plus(days_employed):
    if days_employed < 0: # если трудовой стаж меньше 0, тогда умножаем на константу, которая равна -1
        return days_employed * constant
    return days_employed
data['days_employed'] = data['days_employed'].apply(days_employed_plus)
print('---------------')
print('Минимальное значение трудового стажа:',data['days_employed'].min())
print('Максимальное значение трудового стажа(из таблицы до корректировки):',data['days_employed'].max())
print('Максимальный возраст:', data['dob_years'].max())

max_days_employed = (75 - 14)*247 #расчитаем максимальный трудовой стаж в днях,
# где 75 лет  - окончание работы (максимальный возраст из выборки), 
#14 лет - начало работы (согласно законодательству) и умноженной на 247 рабочих дней в году
max_days_employed_hour = max_days_employed*8
median_days_employed = data['days_employed'].median()
print('Максимальный трудовой стаж (рассчитаный):', max_days_employed , 'дней или', max_days_employed_hour, 'часов' )
print('Медиана трудового стажа:', median_days_employed , 'дней')

# Напишем функцию, которая если стаж из таблицы превышает максимальный рассчитаный стаж в часах,
# то возвращает значение медианы по стажу.
# Если стаж больше максимального стажа в днях, то мы делим значение стажа на 8 часов.
# Т.к. здесь вводили данные в часах, а не днях.

def days_employed_max(days_employed): 
    if days_employed > max_days_employed_hour: 
        return median_days_employed
    if days_employed > max_days_employed: 
        return days_employed / 8
    return days_employed
data['days_employed'] = data['days_employed'].apply(days_employed_max)
print('Максимальный трудовой стаж (после обработки данных):',data['days_employed'].max())
print(data.head(10))

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

Медиана дохода: 135514.0

---------------
Кол-во детей по возрастам до обработки неверных значений:
 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
---------------
Кол-во детей по возрастам после обработки неверных значений:
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64
---------------
Минимальное значение трудового стажа: 0.0
Максимальное значение трудового стажа(из таблицы до корректировки): 401755.40047533
Максимальный возраст: 75
Максимальный трудовой стаж (рассчитаный): 15067 дней или 120536 часов
Медиана трудового стажа: 1808.0534339280625 дней
Максимальный трудовой стаж (после обработки данных): 14920.049805042863
   c

### Вывод

    В таблице найдены 47 отрицательных значения по количеству детей, предполагаю что была допущена опечатка и количество детей было введено с отрицательным значение. Также найдено 76 строк с коичеством детей 20, что считаю не коректным , поэтому отрицательные значения были преобразованы в положительные, а кол-во детей 20 было переведно в 2. 
    Найденные пропуски были удалены с помощью функции Loc и fillna и данные проверены с помощью функции isnull() и дальнейшим суммированием sum().
    В трудовом стаже найдены значения, которые превышают  максимальный трудовой стаж. Некоторые значения трудового стажа были введены в часах, хотя должны быть в днях. В связи с чем, такие значения были поделены на 8 часов, но если и это значение превышало максимально допумтимое значение трудового стажа, то в яйчеку возвращалось значение медианы по стажу.
    В таблице с доходом найдены пустые значения, в результате было проведена обработка данных, и в случае если у пенсионера доход равнялся нулю, то в ячейку возвращается среднее значение пенсии в России в размере 14924 руб. В случае если у компаньона, сотрудника, предпренимателя и госслужащий доход равен нулю, тогда это значение заменяется на рассчитанное по медиане. В случае если у студента не указан доход, то значение заменяется на среднюю стипендию в России за 2020 год, которая сотавляет 2980 руб. В случае безработного, то неуказанная сумма заменяется на ноль, а в случае клиента в декрете, неуказанное значение заменяется на максимальное значение пособие по уходу за ребенком в России за 2020 год, которое состовляет 27985 руб.

### Замена типа данных

In [19]:
print('ОБЩАЯ ИНФОРМАЦИЯ О ТАБЛИЦЕ ДО ЗАМЕНЫ ТИПА ДАННЫХ:')
data.info()
data['total_income'] = data['total_income'].astype('int') # в столбце 'total_income' применив функцию astype()
data['days_employed'] = data['days_employed'].astype('int') #заменим дип данных float64 на int64 
# в столбце 'days_employed' применив функцию astype()
print('------------------------------------')
print('ОБЩАЯ ИНФОРМАЦИЯ О ТАБЛИЦЕ ПОСЛЕ ЗАМЕНЫ ТИПА ДАННЫХ:')
data.info()

ОБЩАЯ ИНФОРМАЦИЯ О ТАБЛИЦЕ ДО ЗАМЕНЫ ТИПА ДАННЫХ:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB
------------------------------------
ОБЩАЯ ИНФОРМАЦИЯ О ТАБЛИЦЕ ПОСЛЕ ЗАМЕНЫ ТИПА ДАННЫХ:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64

### Вывод

Так как в столбцах 'total_income' и 'days_employed' используется тип данных float64 (значения с плавующей запятой), а для рассчетов требуется целочисленные данный(integer), было произведена замена типов данных на int64 c помощью функции astype(). 

### Обработка дубликатов

In [20]:
# УДАЛИМ ДУБЛИКАТЫ
print ('Дубликатов в таблице до удаления:', data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop = True)
print ('Дубликатов в таблице восле удаления:', data.duplicated().sum())

# РАССМОТРИМ ДУБЛИКАТЫ ПО СТОЛБЦАМ
# Рассмотрим дубликаты в столбце образование ('education')
print('')
print('Таблица со всеми значениями в столбце образование (education):')
print('')
print(data['education'].value_counts())
data['education']= data['education'].str.lower()
#data = data.drop_duplicates().reset_index(drop = True)
print('')
print('Таблица со всеми значениями в столбце образование (education) после удаления дубликатов:')
print('')
print(data['education'].value_counts())

# Рассмотрим дубликаты в столбце образование ('family_status')
print('')
print('Таблица со всеми значениями в столбце family_status:')
print('')
print(data['family_status'].value_counts())
data['family_status']= data['family_status'].str.lower() # в столбце найдены значения, которые начинаются с большой буквы,
#  поэтому требуется привести их к нижнему регистру.
print('')
print('Таблица со всеми значениями в столбце family_status после удаления дубликатов:')
print('')
print(data['family_status'].value_counts())

# Рассмотрим дубликаты в столбце 'income_type'
print('')
print('Таблица со всеми значениями в столбце income_type:')
print('')
print(data['income_type'].value_counts())

# Рассмотрим дубликаты в столбце 'purpose'
print('')
print('Таблица со всеми значениями в столбце income_type:')
print('')
print(data['purpose'].value_counts())
data['purpose'] = data['purpose'].replace('на проведение свадьбы', 'свадьба') # заменим схожие значения одним общим
data['purpose'] = data['purpose'].replace('сыграть свадьбу', 'свадьба')
data['purpose'] = data['purpose'].replace('сыграть свадьбу', 'свадьба')

data['purpose'] = data['purpose'].replace('операции с недвижимостью', 'недвижимость')
data['purpose'] = data['purpose'].replace('покупка коммерческой недвижимости', 'недвижимость')
data['purpose'] = data['purpose'].replace('операции с жильем', 'недвижимость')
data['purpose'] = data['purpose'].replace('покупка жилья для сдачи', 'недвижимость')
data['purpose'] = data['purpose'].replace('операции с коммерческой недвижимостью', 'недвижимость')
data['purpose'] = data['purpose'].replace('покупка жилья', 'недвижимость')
data['purpose'] = data['purpose'].replace('жилье', 'недвижимость')
data['purpose'] = data['purpose'].replace('покупка жилья для семьи', 'недвижимость')
data['purpose'] = data['purpose'].replace('строительство собственной недвижимости', 'недвижимость')
data['purpose'] = data['purpose'].replace('строительство жилой недвижимости', 'недвижимость')
data['purpose'] = data['purpose'].replace('покупка недвижимости', 'недвижимость')
data['purpose'] = data['purpose'].replace('покупка своего жилья', 'недвижимость')
data['purpose'] = data['purpose'].replace('строительство недвижимости', 'недвижимость')
data['purpose'] = data['purpose'].replace('ремонт жилью', 'недвижимость')
data['purpose'] = data['purpose'].replace('покупка жилой недвижимости', 'недвижимость')
data['purpose'] = data['purpose'].replace('операции со своей недвижимостью', 'недвижимость')

data['purpose'] = data['purpose'].replace('на покупку своего автомобиля', 'автомобиль')
data['purpose'] = data['purpose'].replace('сделка с подержанным автомобилем', 'автомобиль')
data['purpose'] = data['purpose'].replace('свой автомобиль', 'автомобиль')
data['purpose'] = data['purpose'].replace('на покупку подержанного автомобиля', 'автомобиль')
data['purpose'] = data['purpose'].replace('автомобили', 'автомобиль')
data['purpose'] = data['purpose'].replace('на покупку автомобиля', 'автомобиль')
data['purpose'] = data['purpose'].replace('приобретение автомобиля', 'автомобиль')
data['purpose'] = data['purpose'].replace('сделка с автомобилем', 'автомобиль')

data['purpose'] = data['purpose'].replace('заняться высшим образованием', 'образование')
data['purpose'] = data['purpose'].replace('дополнительное образование', 'образование')
data['purpose'] = data['purpose'].replace('высшее образование', 'образование')
data['purpose'] = data['purpose'].replace('получение дополнительного образования', 'образование')
data['purpose'] = data['purpose'].replace('получение образования', 'образование')
data['purpose'] = data['purpose'].replace('профильное образование', 'образование')
data['purpose'] = data['purpose'].replace('получение высшего образования', 'образование')
data['purpose'] = data['purpose'].replace('заняться образованием', 'образование')

print('')
print('Таблица со всеми значениями в столбце purpose после удаления дубликатов:')
print('')
print(data['purpose'].value_counts())


Дубликатов в таблице до удаления: 54
Дубликатов в таблице восле удаления: 0

Таблица со всеми значениями в столбце образование (education):

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Таблица со всеми значениями в столбце образование (education) после удаления дубликатов:

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Таблица со всеми значениями в столбце family_status:

женат / замужем          12344
гражданский брак          4163
Не женат / 

# Далее проанализораваны столбцы education', 'family_status', 'income_type', 'purpose'.
    Столбец 'education' содержал одинаковые значения в разных регистрах, поэтому все значения были приведен к нижнему регистру.
    Столбец 'family_status' - содержал значения начинающиеся с заглавной буквы, поэтому все значения были приведены к нижнему регистру.
    Столбец 'income_type' был проверен и он не содержит дубликатов.
    Столбец 'purpose' содержал очень много значений, но все значения можно было объеденить по группам. С помощью замены наименований аналогичных целей - одной, которая отражает их смысл, получилось, что основных целей всего четыре, это недвижимсоть, автомобиль, образование и свадьба. Аналогичное м этим столбцом можно проделать с поиощью леммы. Данные дубликаты возникли в связи с тем, что нет общего правила по заполнению данных, поэтому они заполнялись в разных регистрах и цели на кредит нельзя было выбрать, поэтому появились множество вариаций на скожие цели.

### Вывод

    В таблице присуттсвутю дубликаты, для просмотра количества дубликатов воспользуемся функцией data.duplicated(), которые удаляем с помощью функции drop_duplicates().reset_index(drop = True). Дубликаты могли возникнуть потому что , клиент мог несколько раз обращаться в банк, в связи с чем его данные заводились несколько раз. 
    Далее проанализораваны столбцы education', 'family_status', 'income_type', 'purpose'.
    Столбец 'education' содержал одинаковые значения в разных регистрах, поэтому все значения были приведен к нижнему регистру.
    Столбец 'family_status' - содержал значения начинающиеся с заглавной буквы, поэтому все значения были приведены к нижнему регистру.
    Столбец 'income_type' был проверен и он не содержит дубликатов.
    Столбец 'purpose' содержал очень много значений, но все значения можно было объеденить по группам. С помощью замены наименований аналогичных целей - одной, которая отражает их смысл, получилось, что основных целей всего четыре, это недвижимсоть, автомобиль, образование и свадьба. Аналогичное м этим столбцом можно проделать с поиощью леммы. Данные дубликаты возникли в связи с тем, что нет общего правила по заполнению данных, поэтому они заполнялись в разных регистрах и цели на кредит нельзя было выбрать, поэтому появились множество вариаций на скожие цели.

    

### Лемматизация

In [21]:
from pymystem3 import Mystem # импортируем библиотеку с функцией лемматизации на русском языке — pymystem3
m = Mystem()

def lemma(data):
    lemmas = m.lemmatize(data) #напишем функцию которая лемматизирует данные
    return lemmas

data['lemmas'] = data['purpose'].apply(lemma) #применим функцию к столбцу 'lemmas' в data
print('ЛЕММАТИЗИРОВАННАЯ ТАБЛИЦА ПО СТОЛБЦУ PURPOSE:')
data.head(4)

ЛЕММАТИЗИРОВАННАЯ ТАБЛИЦА ПО СТОЛБЦУ PURPOSE:


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,"[недвижимость, \n]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,"[автомобиль, \n]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,"[недвижимость, \n]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,"[образование, \n]"


### Вывод

Данные были лемматизированы по столбцу 'цель получения кредита'. И на основание этих данных можно будет проанализировать зависимость выплаты кредита в срок от цели кредита. Также явно видно, что основных целей кредита четыре, это покупка недвижимости или автомобиля, проведение свадьбы или получения образования.

### Категоризация данных

In [22]:
data.info()# посмотрим данные о таблице и выберим необходимые столбцы. Далее создадим новую таблицу
data_category = data[['children','family_status','family_status_id','debt','total_income','purpose', 'lemmas']]
print(data_category.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 13 columns):
children            21471 non-null int64
days_employed       21471 non-null int64
dob_years           21471 non-null int64
education           21471 non-null object
education_id        21471 non-null int64
family_status       21471 non-null object
family_status_id    21471 non-null int64
gender              21471 non-null object
income_type         21471 non-null object
debt                21471 non-null int64
total_income        21471 non-null int64
purpose             21471 non-null object
lemmas              21471 non-null object
dtypes: int64(7), object(6)
memory usage: 2.1+ MB
   children     family_status  family_status_id  debt  total_income  \
0         1   женат / замужем                 0     0        253875   
1         1   женат / замужем                 0     0        112080   
2         0   женат / замужем                 0     0        145885   
3         3   жена

### Вывод

Создадим новую таблицу, которая необходима для получения выводов по задачам. Оставим в данной таблице только столбwы с 'количеством детей', 'семейным положением', 'идентификатором семейного положения', 'имел ли задолженность по возврату кредитов', 'ежемесячного дохода, 'целью кредита' и 'лемматезированой целью кредита'.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [23]:
# Напишем функцию, которая посчитает количество клиентов с/без детей с условием, что есть/нет долгов
def data_category_children(row):
    children_have = row['children']
    debt_have = row['debt']
    if children_have >= 1:
        if debt_have == 0:
            return 'Есть дети и нет долгов'
    if children_have < 1:
        if debt_have == 1:
            return 'Есть дети и есть долги'
    if children_have < 1:
        if debt_have == 0:
            return 'Нет детей и нет долгов'
    return 'Нет детей и есть долги'

data['children_status'] = data.apply(data_category_children, axis=1)
print('КОЛИЧЕСТВО КЛИЕНТОВ ПО КАТЕГОРИЯМ:')
print(data['children_status'].value_counts())

# Вычислим процент клиентов не оплачивающих кредит вовремя в зависимости от наличия детей
Cat1_child = data.loc[data.loc[:,'children_status'] == 'Есть дети и нет долгов']['children_status'].count()
Cat2_child = data.loc[data.loc[:,'children_status'] == 'Есть дети и есть долги']['children_status'].count()
Cat3_no_child = data.loc[data.loc[:,'children_status'] == 'Нет детей и нет долгов']['children_status'].count()
Cat4_no_child = data.loc[data.loc[:,'children_status'] == 'Нет детей и есть долги']['children_status'].count()

Cat_with_child = Cat2_child / (Cat1_child + Cat2_child) * 100
print('-----------------------------------------')
print('Процент клиентов с детьми, которые не оплачивают кредит в срок:','{:.1f}'.format(Cat_with_child),'%')

Cat_without_child = Cat4_no_child / (Cat3_no_child + Cat4_no_child) * 100
print('Процент клиентов без детей, которые не оплачивают кредит в срок:', '{:.1f}'.format(Cat_without_child),'%')

КОЛИЧЕСТВО КЛИЕНТОВ ПО КАТЕГОРИЯМ:
Нет детей и нет долгов    13044
Есть дети и нет долгов     6686
Есть дети и есть долги     1063
Нет детей и есть долги      678
Name: children_status, dtype: int64
-----------------------------------------
Процент клиентов с детьми, которые не оплачивают кредит в срок: 13.7 %
Процент клиентов без детей, которые не оплачивают кредит в срок: 4.9 %


### Вывод

В категории клиентов с детьми кредит не оплачивают во время 13.7%, что на 8.8 % больше, чем в категории клиентов безе детей и составляет 4.9%.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [24]:
# Напишем функцию, которая будет возвращать значения семейного положения в зависимсоти
# от наличия / не наличия долгов 
def data_category_family(row):
    family_status_have = row['family_status']
    debt_have = row['debt']
    if family_status_have == 'женат / замужем':
        if debt_have == 0:
            return 'женат / замужем и нет долгов'
    if family_status_have == 'женат / замужем':
        if debt_have == 1:
            return 'женат / замужем и есть долги'
    if family_status_have == 'гражданский брак':
        if debt_have == 0:
            return 'гражданский брак и нет долгов'
    if family_status_have == 'гражданский брак':
        if debt_have == 1:
            return 'гражданский брак и есть долги'
    if family_status_have == 'вдовец / вдова':
        if debt_have == 0:
            return 'вдовец / вдова и нет долгов'
    if family_status_have == 'вдовец / вдова':
        if debt_have == 1:
            return 'вдовец / вдова и есть долги'
    if family_status_have == 'в разводе':
        if debt_have == 0:
            return 'в разводе и нет долгов'
    if family_status_have == 'в разводе':
        if debt_have == 1:
            return 'в разводе и есть долги'
    if family_status_have == 'не женат / не замужем':
        if debt_have == 0:
            return 'Не женат / не замужем и нет долгов'
    if family_status_have == 'не женат / не замужем':
        if debt_have == 1:
            return 'Не женат / не замужем и есть долги'

data['family_status_cat'] = data.apply(data_category_family, axis=1)
print('Общая таблица по каждой категории семейного положения:')
print()
print(data['family_status_cat'].value_counts())

# Вычислим процент клиентов не оплачивающих кредит во время в каждой категории семейного положения
Cat1_family = data.loc[data.loc[:,'family_status_cat'] == 'женат / замужем и нет долгов']['family_status_cat'].count()
Cat2_family = data.loc[data.loc[:,'family_status_cat'] == 'женат / замужем и есть долги']['family_status_cat'].count()
Cat_family_id1 = Cat2_family /(Cat1_family + Cat2_family) * 100

Cat3_family = data.loc[data.loc[:,'family_status_cat'] == 'гражданский брак и нет долгов']['family_status_cat'].count()
Cat4_family = data.loc[data.loc[:,'family_status_cat'] == 'гражданский брак и есть долги']['family_status_cat'].count()
Cat_family_id2 = Cat4_family / (Cat3_family + Cat4_family) * 100

Cat5_family = data.loc[data.loc[:,'family_status_cat'] == 'вдовец / вдова и нет долгов']['family_status_cat'].count()
Cat6_family = data.loc[data.loc[:,'family_status_cat'] == 'вдовец / вдова и есть долги']['family_status_cat'].count()
Cat_family_id3 = Cat6_family / (Cat5_family + Cat6_family) * 100

Cat7_family = data.loc[data.loc[:,'family_status_cat'] == 'в разводе и нет долгов']['family_status_cat'].count()
Cat8_family = data.loc[data.loc[:,'family_status_cat'] == 'в разводе и есть долги']['family_status_cat'].count()
Cat_family_id4 = Cat8_family / (Cat7_family + Cat8_family) * 100

Cat9_family = data.loc[data.loc[:,'family_status_cat'] == 'Не женат / не замужем и нет долгов']['family_status_cat'].count()
Cat10_family = data.loc[data.loc[:,'family_status_cat'] == 'Не женат / не замужем и есть долги']['family_status_cat'].count()
Cat_family_id5 = Cat10_family / (Cat9_family + Cat10_family) * 100

print('')
print('ПРОЦЕНТ КЛИЕНТОВ ИМЕЮЩИЕ ДОЛГИ ПО КРЕДИТУ В КАЖДОЙ КАТЕГОРИИ СЕМЕЙНОГО ПОЛОЖЕНИЯ:')
print('Клиенты женатые / замужем, которые не оплачивают кредит в срок:','{:.1f}'.format(Cat_family_id1),'%')
print('Клиенты в гражданском браке, которые не оплачивают кредит в срок:','{:.1f}'.format(Cat_family_id2),'%')
print('Клиенты вдовец / вдова, которые не оплачивают кредит в срок:','{:.1f}'.format(Cat_family_id3),'%')
print('Клиенты в разводе, которые не оплачивают кредит в срок:','{:.1f}'.format(Cat_family_id4),'%')
print('Клиенты не женатые / не замужем, которые не оплачивают кредит в срок:','{:.1f}'.format(Cat_family_id5),'%')

Общая таблица по каждой категории семейного положения:

женат / замужем и нет долгов          11413
гражданский брак и нет долгов          3775
Не женат / не замужем и нет долгов     2536
в разводе и нет долгов                 1110
женат / замужем и есть долги            931
вдовец / вдова и нет долгов             896
гражданский брак и есть долги           388
Не женат / не замужем и есть долги      274
в разводе и есть долги                   85
вдовец / вдова и есть долги              63
Name: family_status_cat, dtype: int64

ПРОЦЕНТ КЛИЕНТОВ ИМЕЮЩИЕ ДОЛГИ ПО КРЕДИТУ В КАЖДОЙ КАТЕГОРИИ СЕМЕЙНОГО ПОЛОЖЕНИЯ:
Клиенты женатые / замужем, которые не оплачивают кредит в срок: 7.5 %
Клиенты в гражданском браке, которые не оплачивают кредит в срок: 9.3 %
Клиенты вдовец / вдова, которые не оплачивают кредит в срок: 6.6 %
Клиенты в разводе, которые не оплачивают кредит в срок: 7.1 %
Клиенты не женатые / не замужем, которые не оплачивают кредит в срок: 9.8 %


### Вывод

Из полученных данных видно, что клиенты в гражданском браке и не женатые/не замужем не оплачивают кредит во время в 9.3% и 9.8% случаев и это максимальные значения. Таким образом не семейные клиенты чаще не оплачивают кредит во время, но отличие здесь не большое, колеблиться в диапозоне 2% - 3%.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [25]:
# Посмотрим средний, максимальный, минимальный доход клиентов, а также вычислим медиану.
print('Средний доход:',data['total_income'].mean()) 
print('Максимальный доход:',data['total_income'].max())
print('Минимальный доход:',data['total_income'].min())
print('Медиана дохода:',data['total_income'].median())

# напишем функцию, которая разделит все значения дохода на три категории:
# низкий доход, средний доход и высокий доход. На основе полученных данных
# по максимальному, среднему, минимальному доходу и медиане,
# примим, что доход менее 40 000 руб. - это низкий доход
# доход меньше медианы (135 716 руб) и до 40 000 - это средний доход,
# доход выше медианы - это высокий доход.
def total_income_stat(total_income): 
        if total_income <= 40000:
                return 'низкий доход'
        if total_income <= 135716:
                return 'средний доход'
        return 'высокий доход' 
data['total_income_stat'] = data['total_income'].apply(total_income_stat)

# Напишем функцию, которая будет возвращать значения дохода в зависимсоти
# от наличия / не наличия  долгов 
def total_income_debt_stat(row):
    total_income_stat = row['total_income_stat']
    debt_have = row['debt']
    if total_income_stat == 'высокий доход':
        if debt_have == 0:
            return 'Высокий доход и нет долгов'
    if total_income_stat == 'высокий доход':
        if debt_have == 1:
            return 'Высокий доход и есть долги'
    if total_income_stat == 'средний доход':
        if debt_have == 0:
            return 'Средний доход и нет долгов'
    if total_income_stat == 'средний доход':
        if debt_have == 1:
            return 'Средний доход и есть долги'
    if total_income_stat == 'низкий доход':
        if debt_have == 0:
            return 'Низкий доход и нет долгов'
    if total_income_stat == 'низкий доход':
        if debt_have == 1:
            return 'Низкий доход и есть долги'
print('')
print('Общая таблица по каждой категории дохода:')
print('')
data['total_income_debt_stat'] = data.apply(total_income_debt_stat, axis=1)
print(data['total_income_debt_stat'].value_counts())

# Вычислим процент клиентов не оплачивающих кредит во время в каждой категории дохода
Cat1_income = data.loc[data.loc[:,'total_income_debt_stat'] == 'Высокий доход и нет долгов']['total_income_debt_stat'].count()
Cat2_income = data.loc[data.loc[:,'total_income_debt_stat'] == 'Высокий доход и есть долги']['total_income_debt_stat'].count()
Cat_income_id1 = Cat2_income / (Cat1_income + Cat2_income) * 100

Cat3_income = data.loc[data.loc[:,'total_income_debt_stat'] == 'Средний доход и нет долгов']['total_income_debt_stat'].count()
Cat4_income = data.loc[data.loc[:,'total_income_debt_stat'] == 'Средний доход и есть долги']['total_income_debt_stat'].count()
Cat_income_id2 = Cat4_income / (Cat3_income + Cat4_income) * 100

Cat5_income = data.loc[data.loc[:,'total_income_debt_stat'] == 'Низкий доход и нет долгов']['total_income_debt_stat'].count()
Cat6_income = data.loc[data.loc[:,'total_income_debt_stat'] == 'Низкий доход и есть долги']['total_income_debt_stat'].count()
Cat_income_id3 = Cat6_income / (Cat5_income + Cat6_income) * 100

print('')
print('Процент клиентов имеющих долги по каждой категории дохода:')
print('Клиенты с высоким доходом, которые не оплачивают кредит в срок:','{:.1f}'.format(Cat_income_id1),'%')
print('Клиенты со средним доходом, которые не оплачивают кредит в срок:','{:.1f}'.format(Cat_income_id2),'%')
print('Клиенты с низким доходом, которые не оплачивают кредит в срок:','{:.1f}'.format(Cat_income_id3),'%')

Средний доход: 150890.93591355783
Максимальный доход: 2265604
Минимальный доход: 0
Медиана дохода: 135716.0

Общая таблица по каждой категории дохода:

Высокий доход и нет долгов    9868
Средний доход и нет долгов    7797
Низкий доход и нет долгов     2065
Высокий доход и есть долги     867
Средний доход и есть долги     694
Низкий доход и есть долги      180
Name: total_income_debt_stat, dtype: int64

Процент клиентов имеющих долги по каждой категории дохода:
Клиенты с высоким доходом, которые не оплачивают кредит в срок: 8.1 %
Клиенты со средним доходом, которые не оплачивают кредит в срок: 8.2 %
Клиенты с низким доходом, которые не оплачивают кредит в срок: 8.0 %


### Вывод

Согласно полученным данным можно сделать вывод, что нет зависимости оплаты кредиты в срок от получаемого доходы.

- Как разные цели кредита влияют на его возврат в срок?

In [26]:
# Создадим функцию, которая будет обобщать схожие значения леммы и возвращать одно значение, 
# например для леммы "недвижимость" и "жилье" возвращать значение 'операции с недвижимостью'
from pymystem3 import Mystem
m = Mystem()
def get_lemma(row):
    lemman = m.lemmatize(row['purpose'])
    if 'недвижимость' in lemman:
        return 'операции с недвижимостью'
    if 'жилье'in lemman:
        return 'операции с недвижимостью'
    if 'свадьба' in lemman:
         return 'проведение свадьбы'
    if 'образование' in lemman:
        return 'образование'
    if 'автомобиль' in lemman:
        return 'покупка автомобиля'
    return 'другое'
data['lemma_classif'] = data.apply(get_lemma,axis=1)

# Напишем функцию, которая будет возвращать цель кредита в зависимосоти
# от наличия / не наличия  долгов 
def purpose_debt(row):
    purpose_stat = row['lemma_classif']
    debt_have = row['debt']
    
    if purpose_stat == 'операции с недвижимостью':
        if debt_have == 0:
            return 'Операции с недвижимостью и нет долгов'
    if purpose_stat == 'операции с недвижимостью':
        if debt_have == 1:
            return 'Операции с недвижимостью и есть долги'
        
    if purpose_stat == 'проведение свадьбы':
        if debt_have == 0:
            return 'Проведение свадьбы и нет долгов'
    if purpose_stat == 'проведение свадьбы':
        if debt_have == 1:
            return 'Проведение свадьбы и есть долги'
        
    if purpose_stat == 'образование':
        if debt_have == 0:
            return 'Образование и нет долгов'
    if purpose_stat == 'образование':
        if debt_have == 1:
            return 'Образование и есть долги'
        
    if purpose_stat == 'покупка автомобиля':
        if debt_have == 0:
            return 'Покупка автомобиля и нет долгов'
    if purpose_stat == 'покупка автомобиля':
        if debt_have == 1:
            return 'Покупка автомобиля и есть долги'
        
    if purpose_stat == 'другое':
        if debt_have == 0:
            return 'Другое и нет долгов'
    if purpose_stat == 'другое':
        if debt_have == 1:
            return 'Другое и есть долги'

print('')
print('Общая таблица по каждой цели кредита:')
print('')
data['purpose_debt'] = data.apply(purpose_debt, axis=1)
print(data['purpose_debt'].value_counts())

# Вычислим процент клиентов не оплачивающих кредит воремя в зависимсоти от цели кредита
Cat1_purpose = data.loc[data.loc[:,'purpose_debt'] == 'Операции с недвижимостью и нет долгов']['purpose_debt'].count()
Cat2_purpose = data.loc[data.loc[:,'purpose_debt'] == 'Операции с недвижимостью и есть долги']['purpose_debt'].count()
Cat_purpose_id1 = Cat2_purpose / (Cat1_purpose + Cat2_purpose) * 100

Cat3_purpose = data.loc[data.loc[:,'purpose_debt'] == 'Проведение свадьбы и нет долгов']['purpose_debt'].count()
Cat4_purpose = data.loc[data.loc[:,'purpose_debt'] == 'Проведение свадьбы и есть долги']['purpose_debt'].count()
Cat_purpose_id2 = Cat4_purpose / (Cat3_purpose + Cat4_purpose) * 100

Cat5_purpose = data.loc[data.loc[:,'purpose_debt'] == 'Образование и нет долгов']['purpose_debt'].count()
Cat6_purpose = data.loc[data.loc[:,'purpose_debt'] == 'Образование и есть долги']['purpose_debt'].count()
Cat_purpose_id3 = Cat6_purpose / (Cat5_purpose + Cat6_purpose) * 100

Cat7_purpose = data.loc[data.loc[:,'purpose_debt'] == 'Покупка автомобиля и нет долгов']['purpose_debt'].count()
Cat8_purpose = data.loc[data.loc[:,'purpose_debt'] == 'Покупка автомобиля и есть долги']['purpose_debt'].count()
Cat_purpose_id4 = Cat8_purpose / (Cat7_purpose + Cat8_purpose) * 100

print('')
print('Процент клиентов имеющих долги по каждой цели кредита:')
print('Клиенты с кредитом на недвижимость, которые не оплачивают кредит в срок:','{:.1f}'.format(Cat_purpose_id1),'%')
print('Клиенты с кредитом на свадьбу, которые не оплачивают кредит в срок:','{:.1f}'.format(Cat_purpose_id2),'%')
print('Клиенты с кредитом на образование, которые не оплачивают кредит в срок:','{:.1f}'.format(Cat_purpose_id3),'%')
print('Клиенты с кредитом на автомобиль, которые не оплачивают кредит в срок:','{:.1f}'.format(Cat_purpose_id4),'%')


Общая таблица по каждой цели кредита:

Операции с недвижимостью и нет долгов    10032
Покупка автомобиля и нет долгов           3905
Образование и нет долгов                  3644
Проведение свадьбы и нет долгов           2149
Операции с недвижимостью и есть долги      782
Покупка автомобиля и есть долги            403
Образование и есть долги                   370
Проведение свадьбы и есть долги            186
Name: purpose_debt, dtype: int64

Процент клиентов имеющих долги по каждой цели кредита:
Клиенты с кредитом на недвижимость, которые не оплачивают кредит в срок: 7.2 %
Клиенты с кредитом на свадьбу, которые не оплачивают кредит в срок: 8.0 %
Клиенты с кредитом на образование, которые не оплачивают кредит в срок: 9.2 %
Клиенты с кредитом на автомобиль, которые не оплачивают кредит в срок: 9.4 %


### Вывод

Согласно полученным данным, клиенты с кредитом на образование и на покупку автомобиля чаще всего не оплачивают кредит в срок - 10.2% и 10.3% соответсвенно. Клиенты берущие кредит на свадьбу не платят в срок в - 7.8% случаев, а в случае с кредитами на недвижимость - в 8.7%.

# Вернул первый вариант и исправил вычислительную ошибку ошибку во всех 4-х случаях.

### Шаг 4. Общий вывод

Зависимость между категорией клиента берущего кредит и не погашением его в срок существует и она сильно отличается. Самый высокий процент не оплаты кредита в срок, в категории клиентов с детьми. Из этой категории клиентов 13.7% не платят кредит в срок, в отличие от категории клиентов без детей, тут не платят в срок - 4.9%. Также клиенты не женатые/ не замужем и в гражданском браке, тут значения 9.8% и 9.3%, в остальных категориях относительно семейного положения показатели находятся в промежутке 6.6% - 7.5%. Уровень дохода не зависит на ответственость клиентов перед банком по оплате кредита и здесь все категории находятся в очень узком диапозоне 8.0% - 8.2%. Касательно цели кредита, наблюдается зависимость, что при покупке недвижимости, оплата кредита задерживает минимальное количество клиентов - 7.2%, при остальных целях процент не оплаты в срок растет, при кредите на свадьбу - это 8.0%, а при покупке автомобиля и получении образования - 9.4% и 9.2%.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.
- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод. (edited) 